In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install foolbox

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
     |████████████████████████████████| 62 kB 733 kB/s 
     |████████████████████████████████| 170 kB 32.6 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from __future__ import print_function
import torch
import numpy as np
import warnings
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datetime import datetime
import torchvision.models as models
import urllib
import os
import math
import torch.nn.init as init
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD,LinfBasicIterativeAttack,LinfFastGradientAttack,L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack

In [ ]:
__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]

In [ ]:
class VGG(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [ ]:
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


In [ ]:
cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
          512, 512, 512, 512, 'M'],
}

In [ ]:
def vgg19_bn():
    """VGG 19-layer model (configuration 'E') with batch normalization"""
    return VGG(make_layers(cfg['E'], batch_norm=True))


In [ ]:

transform = transforms.ToTensor()

In [ ]:
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]

norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
mc_dropout_samples = 30


In [ ]:

test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='data', train=False, transform=transform, download=True),
        batch_size=64, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


In [ ]:
def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

In [ ]:
def fgsm(image,model,epsilon, label):

    image = image.detach()

    item_count = image.shape[0]

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()
    o = model(norm((image + delta)))
    loss = nn.CrossEntropyLoss(reduce=False)(o, label)
    loss = loss.reshape(1, item_count)
    
    loss.backward(torch.ones_like(loss))

    delta.data = delta + epsilon * delta.grad.detach().sign()
    delta.grad.zero_()

    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [ ]:

def rectified_fgsm(image,model,epsilon):

    torch.manual_seed(2)
    torch.cuda.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(norm(image))
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    dropout_predictions = torch.zeros([mc_dropout_samples,item_count,10])

    for i in range(mc_dropout_samples):

        enable_dropout(model)
        output = model(norm((image + delta)))
        output = softmax(output)

        dropout_predictions[i] = output

    variance = torch.var(dropout_predictions, dim=0)

    var = variance.mean(1,True)
    var = var.reshape(1,item_count)
    var = var.to(device)

    model.eval()
    o = model(norm((image + delta)))
    loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
    loss = loss.reshape(1, item_count)

    loss.backward(torch.ones_like(var))
    delta_loss = delta.grad.detach().sign()
    delta.grad.zero_()
    delta.grad = None

    var.backward(torch.ones_like(var))
    delta_unc = delta.grad.detach().sign()
    delta.grad.zero_()
    delta.grad = None

    zeros = torch.zeros_like(delta_loss)
    delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
    delta.data = (delta + epsilon * delta_unc)

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [ ]:
def bim(image,model,epsilon, label, alpha, num_iter):

    image = image.detach()

    item_count = image.shape[0]

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    for t in range(num_iter):

        model.eval()
        o = model(norm((image + delta)))

        loss = nn.CrossEntropyLoss(reduce=False)(o, label)
        loss = loss.reshape(1, item_count)

        loss.backward(torch.ones_like(loss))

        delta.data = (delta + alpha * delta.grad.detach().sign()).clamp(-epsilon, epsilon)
        delta.grad.zero_()


    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [ ]:

def rectified_bim(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    torch.cuda.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(norm(image))
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([mc_dropout_samples,item_count,10])

        for i in range(mc_dropout_samples):

            enable_dropout(model)
            output = model(norm((image + delta)))
            output = softmax(output)

            dropout_predictions[i] = output


        variance = torch.var(dropout_predictions, dim=0)

        var = variance.mean(1,True)
        var = var.reshape(1,item_count)
        var = var.to(device)

        model.eval()
        o = model(norm((image + delta)))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
            delta.data = (delta + alpha * delta_unc).clamp(-epsilon, epsilon)

        else:

            model.eval()
            with torch.no_grad():
                o = model(norm((image + delta)))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

##########################################################################################

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp, retain_graph=True)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp, retain_graph=True)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_loss = torch.where(delta_unc != delta_loss, zeros, delta_loss)
            delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

##########################################################################################

            inds_match = np.where(inter_pred.cpu() == init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_match] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_loss = torch.where(delta_unc == delta_loss, zeros, delta_loss)
            delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

##########################################################################################

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [ ]:
def pgd(image,model,epsilon, label, alpha, num_iter, restarts):

    torch.manual_seed(2)
    torch.cuda.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]
    image = image.detach()

    max_loss = torch.zeros(label.shape[0]).to(label.device)
    max_delta = torch.zeros_like(image)

    for i in range(restarts):

        delta = torch.rand_like(image, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon

        for t in range(num_iter):

            model.eval()
            o = model(norm((image + delta)))

            loss = nn.CrossEntropyLoss(reduce=False)(o, label)
            loss = loss.reshape(1, item_count)

            loss.backward(torch.ones_like(loss))

            delta.data = (delta + alpha * delta.grad.detach().sign()).clamp(-epsilon, epsilon)
            delta.grad.zero_()
        
        all_loss = nn.CrossEntropyLoss(reduction='none')(model(norm((image + delta))),label)
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)

    perturbed_image = image + max_delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [ ]:

def rectified_pgd(image, model, epsilon, label, alpha, num_iter, restarts):

    torch.manual_seed(2)
    torch.cuda.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    max_loss = torch.zeros(label.shape[0]).to(label.device)
    max_delta = torch.zeros_like(image)

    for r in range(restarts):

        delta = torch.rand_like(image, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon

        model.eval()
        with torch.no_grad():
            o = model(norm(image))
            o = softmax(o)
        init_pred = o.data.max(1, keepdim=True)[1]

        lbls = torch.squeeze(init_pred,1)

        enable_dropout(model)

        for t in range(num_iter):

            dropout_predictions = torch.zeros([mc_dropout_samples,item_count,10])

            for i in range(mc_dropout_samples):

                enable_dropout(model)
                output = model(norm((image + delta)))
                output = softmax(output)

                dropout_predictions[i] = output

            variance = torch.var(dropout_predictions, dim=0)

            var = variance.mean(1,True)
            var = var.reshape(1,item_count)
            var = var.to(device)

            model.eval()
            o = model(norm((image + delta)))
            loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
            loss = loss.reshape(1, item_count)

            if t == 0:

                loss.backward(torch.ones_like(var))
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                var.backward(torch.ones_like(var))
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                zeros = torch.zeros_like(delta_loss)
                delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
                delta.data = (delta + alpha * delta_unc).clamp(-epsilon, epsilon)

            else:

                model.eval()
                with torch.no_grad():
                    o = model(norm((image + delta)))
                    o = softmax(o)
                inter_pred = o.data.max(1, keepdim=True)[1]

    ##########################################################################################

                inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

                temp = torch.ones_like(var)
                temp = temp.cpu().numpy()

                temp[0][inds_notmatch] = 0
                temp = torch.tensor(temp)
                temp = temp.to(device)

                var.backward(temp, retain_graph=True)
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                loss.backward(temp, retain_graph=True)
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                zeros = torch.zeros_like(delta_loss)
                delta_loss = torch.where(delta_unc != delta_loss, zeros, delta_loss)
                delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)


    ##########################################################################################

                inds_match = np.where(inter_pred.cpu() == init_pred.cpu())[0]

                temp = torch.ones_like(var)
                temp = temp.cpu().numpy()

                temp[0][inds_match] = 0
                temp = torch.tensor(temp)
                temp = temp.to(device)

                var.backward(temp)
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                loss.backward(temp)
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                zeros = torch.zeros_like(delta_loss)
                delta_loss = torch.where(delta_unc == delta_loss, zeros, delta_loss)
                delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

    ##########################################################################################

        all_loss = nn.CrossEntropyLoss(reduction='none')(model(norm((image + delta))),label)
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)

    # model.eval()
    perturbed_image = image + max_delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [ ]:
def norms(Z):
    """Compute norms over all but the first dimension"""
    return Z.view(Z.shape[0], -1).norm(dim=1)[:,None,None,None]


In [ ]:
warnings.filterwarnings("ignore")

torch.manual_seed(2)
torch.cuda.manual_seed(2)
np.random.seed(2)

batch_size = 64
eps = 3./255
alpha = 0.4 * eps
num_iter = 5

num_restart = 5

count_correct_after_attack = 0
count_wrong_after_attack = 0


In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model_cnn = vgg19_bn()
model_cnn.load_state_dict(torch.load("/content/gdrive/MyDrive/model_cnn_vgg19bn_cifar10.pt"))
model_cnn.eval()
model_cnn.to(device)

softmax = nn.Softmax(dim=1)

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)


2021-09-13 13:09:28


In [ ]:
for i, (image, label) in enumerate(test_loader):

    image = image.to(device)
    label = label.to(device)

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(norm(image))
        o = softmax(o)

    pred_original = o.data.max(1, keepdim=True)[1]
    pred_original = pred_original.view_as(label)
    inds_correct = np.where(pred_original.cpu() == label.cpu())[0]

    if inds_correct.shape[0] == 0:
        continue

    image = image[inds_correct]
    label = label[inds_correct]

    image = image.to(device)
    label = label.to(device)

    #pert = fgsm(image,model_cnn,eps, label)
    #pert = rectified_fgsm(image, model_cnn, eps)
    #pert = bim(image, model_cnn, eps, label, alpha, num_iter)
    #pert = rectified_bim(image, model_cnn, eps, num_iter, alpha)
    #pert = pgd(image, model_cnn, eps, label, alpha, num_iter, num_restart)
    pert = rectified_pgd(image, model_cnn, eps, label, alpha, num_iter, num_restart)   

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(norm(pert))
        o = softmax(o)
    pred_pert = o.data.max(1, keepdim=True)[1]

    pred_pert = pred_pert.view_as(label)

    inds_correct_after_attack = np.where(pred_pert.cpu() == label.cpu())[0]
    inds_wrong_after_attack = np.where(pred_pert.cpu() != label.cpu())[0]

    inds_correct_after_attack = inds_correct_after_attack.tolist()
    inds_wrong_after_attack = inds_wrong_after_attack.tolist()

    count_correct_after_attack += len(inds_correct_after_attack)
    count_wrong_after_attack += len(inds_wrong_after_attack)


    if i%30 == 0:
      print("Success ratio for the attack is : %{:.2f}".format(100 * count_wrong_after_attack / (count_correct_after_attack + count_wrong_after_attack)))
    

Success ratio for the attack is : %78.85
Success ratio for the attack is : %83.01
Success ratio for the attack is : %83.00
Success ratio for the attack is : %83.38
Success ratio for the attack is : %83.97
Success ratio for the attack is : %83.88


In [ ]:
print("count_wrong_after_attack ", count_wrong_after_attack)
print("count_correct_after_attack ", count_correct_after_attack)

print("Success ratio for the attack is : %{:.2f}".format(
    100 * count_wrong_after_attack / (count_correct_after_attack + count_wrong_after_attack)))

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

count_wrong_after_attack  7204
count_correct_after_attack  1375
Success ratio for the attack is : %83.97
2021-09-13 17:44:12
